<a href="https://colab.research.google.com/github/JayKayNJIT/InnovisionLLC/blob/main/Mistral7llamaindexDemo_fixed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q pypdf
!pip install -q python-dotenv
!pip install -q transformers
!pip install llama-index-llms-llama-cpp
!pip -q install sentence-transformers
!pip install --upgrade langchain
!pip install llama-index-embeddings-langchain
!pip install -q llama-index


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 MB 15.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 61.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 52.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━

In [2]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install  llama-cpp-python --no-cache-dir

In [7]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, ServiceContext


In [8]:
documents = SimpleDirectoryReader("/content/read").load_data()

Ignoring wrong pointing object 2 65536 (offset 0)
Ignoring wrong pointing object 2 65536 (offset 0)


Ignoring wrong pointing object 51 65536 (offset 0)
Ignoring wrong pointing object 51 65536 (offset 0)


Ignoring wrong pointing object 62 65536 (offset 0)
Ignoring wrong pointing object 62 65536 (offset 0)


Ignoring wrong pointing object 120 65536 (offset 0)
Ignoring wrong pointing object 120 65536 (offset 0)


Ignoring wrong pointing object 138 65536 (offset 0)
Ignoring wrong pointing object 138 65536 (offset 0)


Ignoring wrong pointing object 168 65536 (offset 0)
Ignoring wrong pointing object 168 65536 (offset 0)


Ignoring wrong pointing object 182 65536 (offset 0)
Ignoring wrong pointing object 182 65536 (offset 0)


In [5]:
import torch
from llama_index.llms.llama_cpp import LlamaCPP
from llama_index.llms.llama_cpp.llama_utils import (
    messages_to_prompt,
    completion_to_prompt,
)

llm = LlamaCPP(
    # You can pass in the URL to a GGML model to download it automatically
    #model_url='https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.1-GGUF/resolve/main/mistral-7b-instruct-v0.1.Q4_K_M.gguf',
    # optionally, you can set the path to a pre-downloaded model instead of model_url
    model_path='/home/jatink/Google_Colab/LLM/mistral-7b-instruct-v0.1.Q4_K_M.gguf',
    temperature=0.1,
    max_new_tokens=256,
    # llama2 has a context window of 4096 tokens, but we set it lower to allow for some wiggle room
    context_window=3900,
    # kwargs to pass to __call__()
    generate_kwargs={},
    # kwargs to pass to __init__()
    # set to at least 1 to use GPU
    model_kwargs={"n_gpu_layers": -1},
    # transform inputs into Llama2 format
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    verbose=True,
)


llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from /home/jatink/Google_Colab/LLM/mistral-7b-instruct-v0.1.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.1
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv  

In [6]:
from llama_index.embeddings.langchain import LangchainEmbedding
from langchain.embeddings import HuggingFaceEmbeddings

In [7]:
embed_model = LangchainEmbedding(
  HuggingFaceEmbeddings(model_name="thenlper/gte-large")
)


INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: thenlper/gte-large
Load pretrained SentenceTransformer: thenlper/gte-large
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cuda
Use pytorch device_name: cuda


In [8]:
service_context = ServiceContext.from_defaults(
    chunk_size=256,
    llm=llm,
    embed_model=embed_model
)

/tmp/ipykernel_358651/1276886766.py:1: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(


In [9]:
index = VectorStoreIndex.from_documents(documents, service_context=service_context)

In [11]:
query_engine = index.as_query_engine()

In [12]:
while True:
  query=input()
  response = query_engine.query(query)
  print(response)

Who are authors of "DESCRIPTION AND PERFORMANCE OF THE HYBRID III THREE-YEAR-OLD, SIX-YEAR-OLD AND SMALL FEMALE TEST DUMMIES IN RESTRAINT SYSTEM AND OUT-OF-POSITION AIR BAG ENVIRONMENTS"?



llama_print_timings:        load time =   23415.31 ms
llama_print_timings:      sample time =      31.39 ms /   128 runs   (    0.25 ms per token,  4077.47 tokens per second)
llama_print_timings: prompt eval time =   25533.09 ms /   515 tokens (   49.58 ms per token,    20.17 tokens per second)
llama_print_timings:        eval time =   16306.05 ms /   127 runs   (  128.39 ms per token,     7.79 tokens per second)
llama_print_timings:       total time =   42117.75 ms /   642 tokens


 The authors of "DESCRIPTION AND PERFORMANCE OF THE HYBRID III THREE-YEAR-OLD, SIX-YEAR-OLD AND SMALL FEMALE TEST DUMMIES IN RESTRAINT SYSTEM AND OUT-OF-POSITION AIR BAG ENVIRONMENTS" are Roger A. Saul, Howard B. Pritz, Joeseph McFadden, Stanl ey H. B ackaitis, Natio nal Hig hway Traffic Safety Ad ministratio n, Heather Hallenbeck, and Dan Rhule.


KeyboardInterrupt: Interrupted by user